#  
# JSON structured data from tables in scientific papers

## Jsonformer non-token approach for OpenAI
#### (based on: https://github.com/1rgs/jsonformer, https://github.com/martinezpl/jsonformer/tree/add-openai)

In [1]:
import os
import pandas as pd
import json
import time

import openai
from IPython.display import IFrame
from IPython.display import HTML

from LLMsTablesToJson.html_table import get_driver, extract_tableSource, extract_table, quit_driver
from LLMsTablesToJson.jsonformer_non_tokens import JsonformerNoTokens, OpenAIModel, highlight_values

DATA_DIR = "your_path"
chromedriver_path = os.path.join(DATA_DIR, "chromedriver.exe")

openai.api_key = "your_openai_api_key"

## Example: Properties of activated carbons for CO<sub>2</sub> adsorption

### Get table HTML code from DOI

In [11]:
doi = "10.1016/j.jece.2023.109643"

In [16]:
IFrame(src="https://dx.doi.org/" + doi, width=900, height=380)

In [17]:
driver = get_driver(doi, chromedriver_path)
time.sleep(2)
tableSource = extract_tableSource(driver)

[<selenium.webdriver.remote.webelement.WebElement (session="92e205001e17eecc209a6b0362f8f39e", element="F0984E5B6A88A306A3903857D1646985_element_98")>, <selenium.webdriver.remote.webelement.WebElement (session="92e205001e17eecc209a6b0362f8f39e", element="F0984E5B6A88A306A3903857D1646985_element_99")>, <selenium.webdriver.remote.webelement.WebElement (session="92e205001e17eecc209a6b0362f8f39e", element="F0984E5B6A88A306A3903857D1646985_element_100")>, <selenium.webdriver.remote.webelement.WebElement (session="92e205001e17eecc209a6b0362f8f39e", element="F0984E5B6A88A306A3903857D1646985_element_101")>, <selenium.webdriver.remote.webelement.WebElement (session="92e205001e17eecc209a6b0362f8f39e", element="F0984E5B6A88A306A3903857D1646985_element_102")>, <selenium.webdriver.remote.webelement.WebElement (session="92e205001e17eecc209a6b0362f8f39e", element="F0984E5B6A88A306A3903857D1646985_element_103")>, <selenium.webdriver.remote.webelement.WebElement (session="92e205001e17eecc209a6b0362f8f3

In [18]:
table_num = 4
html_table_carbonsCO2ads = extract_table(tableSource, table_num)
quit_driver(driver)

In [19]:
html_table_carbonsCO2ads

'<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">900</td><td

In [20]:
with open("html_table_carbonsCO2ads.txt", "w") as file:
    file.write(html_table_carbonsCO2ads)

### Show table

In [15]:
HTML(html_table_carbonsCO2ads)

Feedstock,Temperature (∘C),Surface area (m2/g),Total Pore Volume (cm3/g),CO2 capacity (mg/g),Ref.
Rambutan peel,500,7.80,0.011,27.83,[100]
Rambutan peel,700,175.84,0.111,56.61,[100]
Rambutan peel,900,569.64,0.313,68.74,[100]
Hickory wood,300,0.10,NA,34.48,[99]
Hickory wood,450,12.90,NA,44.96,[99]
Hickory wood,600,401.00,NA,61.00,[99]
Sugarcane baggasse,300,5.20,NA,38.72,[99]
Sugarcane baggasse,450,13.60,NA,53.83,[99]
Sugarcane baggasse,600,388.30,NA,73.55,[99]
Perilla leaf,700,473.4,0.1,101.728,[103]


### Convert HTML table to JSON format with openAI LLMs

In [16]:
json_schema_carbonsCO2ads = {
    "type": "object",
    "properties": {
        "Activated carbons": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "Feedstock": {"type": "string"},
                    "Temperature (∘C)": {"type": "number"},
                    "Surface area (m2/g)": {"type": "number"},
                    "Total Pore Volume (cm3/g)": {"type": "number"},
                    "CO2 capacity (mg/g)": {"type": "number"},
                },
            },
        },
    },
}

In [17]:
with open('json_schema_carbonsCO2ads.json', 'w') as file:
    json.dump(json_schema_carbonsCO2ads, file)

In [18]:
with open("json_schema_carbonsCO2ads.json") as file:
    json_schema_carbonsCO2ads = json.load(file)
    file.close()

In [19]:
json_schema_carbonsCO2ads

{'type': 'object',
 'properties': {'Activated carbons': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'Feedstock': {'type': 'string'},
     'Temperature (∘C)': {'type': 'number'},
     'Surface area (m2/g)': {'type': 'number'},
     'Total Pore Volume (cm3/g)': {'type': 'number'},
     'CO2 capacity (mg/g)': {'type': 'number'}}}}}}

In [33]:
builder = JsonformerNoTokens(
    model=OpenAIModel("text-davinci-003", debug=False),
    json_schema=json_schema_carbonsCO2ads,
    text=html_table_carbonsCO2ads[:-5500],
    prompt="Generate a object with the following schema extracting the information from the provided table in html code:",
    temperature=0.5,
    debug=True,
    max_array_length=11,
    max_string_token_length=10,
)

print("Generating...")
result_carbonsCO2ads = builder()

Generating...
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#

[generate_number] response  500, "Surface area
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display

[generate_number] response  0.011, "CO
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display-inline 

[generate_string] response  "Rambutan peel", "Temperature (\
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-tri

[generate_number] response  175.84, "Total
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display-inl

...

[generate_string] response  "Perilla leaf", "Temperature (\u
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-tri

[generate_number] response  473.4, "
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display-inline an

[generate_number] response  101.728}, {"Feed
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display-i

[generate_number] response  700, "Surface area
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display

[generate_number] response  0.2, "CO
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="rowsep-1"><th scope="col">Feedstock</th><th scope="col">Temperature (<sup>∘</sup>C)</th><th scope="col">Surface area (m<sup>2</sup>/g)</th><th scope="col">Total Pore Volume (cm<sup>3</sup>/g)</th><th scope="col">CO<sub>2</sub> capacity (mg/g)</th><th scope="col">Ref.</th></tr></thead><tbody><tr><th scope="row">Rambutan peel</th><td class="align-char">500</td><td class="align-char">7.80</td><td>0.011</td><td class="align-char">27.83</td><td><a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#bib100" name="bbib100"><span class="anchor-text">[100]</span></a></td></tr><tr><th scope="row">Rambutan peel</th><td class="align-char">700</td><td class="align-char">175.84</td><td>0.111</td><td class="align-char">56.61</td><td><a class="anchor workspace-trigger u-display-inline an

In [34]:
highlight_values(result_carbonsCO2ads)

{
  Activated carbons: [
    {
      Feedstock: "Rambutan peel",
      Temperature (∘C): 500,
      Surface area (m2/g): 7.8,
      Total Pore Volume (cm3/g): 0.011,
      CO2 capacity (mg/g): 27.83
    },
    {
      Feedstock: "Rambutan peel",
      Temperature (∘C): 700,
      Surface area (m2/g): 175.84,
      Total Pore Volume (cm3/g): 0.111,
      CO2 capacity (mg/g): 56.61
    },
    {
      Feedstock: "Rambutan peel",
      Temperature (∘C): 900,
      Surface area (m2/g): 569.64,
      Total Pore Volume (cm3/g): 0.313,
      CO2 capacity (mg/g): 68.74
    },
    {
      Feedstock: "Hickory wood",
      Temperature (∘C): 300,
      Surface area (m2/g): 0.1,
      Total Pore Volume (cm3/g): 2,
      CO2 capacity (mg/g): 34.48
    },
    {
      Feedstock: "Hickory wood",
      Temperature (∘C): 450,
      Surface area (m2/g): 12.9,
      Total Pore Volume (cm3/g): 2,
      CO2 capacity (mg/g): 44.96
    },
    {
      Feedstock: "Hickory wood",
      Temperature (∘C): 600,
     

In [ ]:
with open('json_table_carbonsCO2ads.json', 'w') as file:
    json.dump(result_carbonsCO2ads, file)